# Brute force solver for a 2x2 Rubik's cube


We use MeepMoops's Py222 implementation:

In [94]:
from py222 import py222

We begin by getting a solved puzzle

In [95]:
s = py222.initState()
print(py222.isSolved(s))
py222.printCube(s)

True
      ┌──┬──┐
      │ 0│ 0│
      ├──┼──┤
      │ 0│ 0│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 4│ 2│ 2│ 1│ 1│ 5│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 3│ 3│
      ├──┼──┤
      │ 3│ 3│
      └──┴──┘


Next, we can apply a sequence of moves to scramble the puzzle:

In [165]:
# scramble = py222.doAlgStr(s, "x y R U' R'")
scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U")
# scramble = py222.doAlgStr(s, "x y R U' R' U' F2 U' R U R' U F2")
print(py222.isSolved(scramble))
py222.printCube(scramble)

False
      ┌──┬──┐
      │ 0│ 1│
      ├──┼──┤
      │ 2│ 2│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 5│ 0│ 4│ 1│ 0│ 2│ 3│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 3│ 3│ 4│ 5│ 0│ 1│ 5│ 4│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 2│ 4│
      ├──┼──┤
      │ 5│ 3│
      └──┴──┘


Next, we brute force a solution.
Note that we can reduce the degrees of freedom to only nine types of moves: 
* Rotations do not affect state, and can be ignored.
* WLOG, moving the left side is equivalent to moving the right side. Thus, instead of 6 turns ((R)ight, (L)eft, (U)p, (D)own, (F)ront, (B)ack), we can use 3 types of turns ((R)ight, (F)ront, and (U)p).
* For each of the three types of turns, we can rotate clockwise ninety degrees, rotate counterclockwise (indicated by a prime (')), or rotate 180 degrees (indicated by a '2')

Note that God's number for a 2x2 Rubik's cube is 11; i.e., any 2x2 puzzle can be solved within 11 moves. This means, using a naive approach, we can bound our run time as follows:
* Our first move has nine possible options
* Subsequent moves have six possible options (WLOG, if we turned the right face on the _n-1_-th move, we won't want to turn right again in any form on the _n_-th move).

Thus, an upper bound for the number of iterations is O(9*6^10) or 544 million iterations for our naive approach.

We can represent the possible moves:

In [166]:
valid_moves = ["R", "R'", "R2", "U", "U'", "U2", "F", "F'", "F2"]

In [182]:
def solve_222(cube):
    cubes_queue = [(cube, [])]

    num_moves = 0
    i = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            return [valid_moves[move] for move in prev_moves]

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
            num_moves = len(prev_moves)

        prev_move = prev_moves[-1] if len(prev_moves) > 0 else 9

        # Enforce not repeating the same side again
        next_moves = list(range(9))[0:(prev_move//3 * 3)] + list(range(9))[(prev_move//3 * 3 + 3):]

        # Perform each possible move, and add each of those cubes to the queue
        for move in next_moves:
            next_cube = py222.doAlgStr(cube, valid_moves[move])
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

This appears very slow, so we can try a different method of hashing the cubes we've already tried.

(Unfortunately, this approach is even slower!... likely due to the str(cube) operations.)

In [183]:
def solve_222_bad(cube):
    cubes_already_tried = set([str(cube)])
    cubes_queue = [(cube, [])]

    num_moves = 0

    while(True):
        # grab the next cube on the queue
        (cube, prev_moves) = cubes_queue.pop(0)

        if py222.isSolved(cube):
            return prev_moves

        if len(prev_moves) > num_moves:
            print("Num moves: " + str(len(prev_moves)))
            num_moves = len(prev_moves)

        # Perform each possible move, and add each of those cubes to the queue
        for move in valid_moves:
            next_cube = py222.doAlgStr(cube, move)
            if str(next_cube) not in cubes_already_tried:
                cubes_queue.append((next_cube, prev_moves + [move]))

In [184]:
solve_222(scramble)

Num moves: 1	 Iter: 1
Num moves: 2	 Iter: 10
Num moves: 3	 Iter: 64
Num moves: 4	 Iter: 388
Num moves: 5	 Iter: 2332
Num moves: 6	 Iter: 13996
Num moves: 7	 Iter: 83980
Num moves: 7	 Iter: 137765


["R'", 'U', 'F2', 'U', 'R', 'U', "R'"]

We now have a naive way to brute force 2x2 solutions.